In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.spatial.distance as sdist

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
product_df = pd.read_csv("product2.csv", sep=';', encoding='latin')
package_df = pd.read_csv("package2.csv", sep=';', encoding='latin')

# 1.

Auscultez les données et présentez un résumé de votre auscultation (nombre d’attributs
pour chaque table, types d’attributs, valeurs manquantes, incohérences intra-attribut, incohérences inter-attribut entre attributs reliés, vraissemblance et interprétabilité des attributs) ;


## product2.csv --> product_df

File Notes
* Package data can be found in the Packages file, linked by the ProductID field.
* Reference code names (translations) are included instead of the codes themselves.
* Fields that have multiple values are identified with an “MV” after their name. Values are concatenated together by a semi-colon “;”.
* If the term NULL appears after an element name, it means there may be records where no value is provided.
* Complete list: www.fda.gov/edrls

* **ProductID** Text/string. - concatenation of the NDCproduct code and SPL documentID; Help prevent duplicate rows when joining the product and package files. 

* **ProductNDC** Text/string. - The labeler code and product code segments of the National Drug Code number, separated by a hyphen.
 
* **ProductTypeName**  Text/string. - Indicates the type of product; corresponds to the “Document Type” of the SPL submission for the listing
 
* **ProprietaryName** Text/string. - (trade name) It is the name of the product chosen by the labeler.
 
* **ProprietaryNameSuffix** *NULL* Text/string. - A value here should be appended to the ProprietaryName to obtain the complete name of the product.
 
* **NonProprietaryName** Text/string. *MV* (16257, object) - Sometimes called the generic name, this is usually the active ingredient(s) of the product.
 
* **DosageFormName** Text/string. (134, object) - The translation of the DosageForm Code submitted by the firm.
 
* **RouteName** Text/string. *MV* (180, object) - The translation of the Route Code submitted by the firm, indicating route of administration.
 
* **StartMarketingDate** Text/string. - Date that the labeler indicates was the start of its marketing of the drug product.
 
* **EndMarketingDate** *NULL* Text/string. - This is the date the product will no longer be available on the market; expiration date of the last lot produced as the EndMarketingDate

* **MarketingCategoryName** Text/string. (10, object) - Potential Marketing Categories
 
* **ApplicationNumber** *NULL* Text/string. (10711, object) - Corresponds to the NDA, ANDA, or BLA number reported by the labeler for products which have the corresponding Marketing Category designated.
 
* **LabelerName** Text/string. (6611, object) - Name of Company corresponding to the labeler code segment of the ProductNDC.
 
* **SubstanceName** Text/string. *MV* (8976, object) - Active ingredient list. Each ingredient name is the preferred term of the UNII code submitted. 
 
* **StrengthNumber** Text/string. *MV* - Strength values (to be used with units below) of each active ingredient, listed in the same order as the SubstanceName field above.
 
* **StrengthUnit** Text/string. *MV* (2391, object) - Units to be used with the strength values above, listed in the same order as the SubstanceName and SubstanceNumber.
 
* **Pharm_Classes** Text/string. *MV* (1285, object) - Reported pharmacological class categories corresponding to the SubstanceNames listed above.
 
* **DEASchedule** Text/string. (4, object) - Assigned DEA Schedule number as reported by the labeler. Values={ CI, CII, CIII, CIV, CV }.

* **NDC_Exclude_Flag**  Text/String. (1, object) - (_Y_, _N_, _E_, _I_) This indicates whether the PACKAGE has been removed/excluded from the NDC Directory for failure to respond to FDA’s requests
    * for correction to deficient or non-compliant submissions (_Y_)
    * the listing certification is expired (_E_)
    * the listing data was inactivated by FDA (_I_)
    * The PACKAGE.XLS and PACKAGE.TXT files only contain listing records where NDC_EXCLUDE_FLAG={ _N_ } **
    * PACKAGES_EXCLUDED.XLS and PACKAGES_EXCLUDED.TXT file contains all listing records with an NDC_EXCLUDE_FLAG={ _Y_, _E_, _I_ }.
* **Listing_Record_Certified_Through** Text/String - This is the date when the listing record will expire if not updated or certified by the firm.

### Remarques

Nous devrons possiblement joindre les basese de données sur **ProductID**

Il est à noter que tous produits ayant une valeur à la variable **EndMarketingDate** présente une valeur NaN à la variable **Listing_Record_Certified_Through**

**NDC_Exclude_Flag** semble non essentiel puisque nous traitons que de ceux qui sont approuvés.

________________________________

In [3]:
print("product_df shape: ", product_df.shape)
print("# of rows: ", product_df.shape[0])
print("# of attributes: ", product_df.shape[1])

product_df shape:  (93238, 20)
# of rows:  93238
# of attributes:  20


In [4]:
product_df.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


In [5]:
product_df.tail()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
93233,99207-465_7578e84a-41ed-498d-8c2b-56a9931679db,99207-465,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,55,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93234,99207-466_7578e84a-41ed-498d-8c2b-56a9931679db,99207-466,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,80,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93235,99207-467_7578e84a-41ed-498d-8c2b-56a9931679db,99207-467,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,105,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93236,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525,HUMAN PRESCRIPTION DRUG,Vanos,NaN,fluocinonide,CREAM,TOPICAL,20060313,NaN,NDA,NDA021758,Valeant Pharmaceuticals North America LLC,FLUOCINONIDE,1,mg/g,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0
93237,99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb,99207-850,HUMAN PRESCRIPTION DRUG,Luzu,NaN,LULICONAZOLE,CREAM,TOPICAL,20131114,NaN,NDA,NDA204153,"Bausch Health US, LLC",LULICONAZOLE,10,mg/g,"Azole Antifungal [EPC],Azoles [CS],Cytochrome ...",NaN,N,20201231.0


In [6]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93238 entries, 0 to 93237
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PRODUCTID                         91678 non-null  object 
 1   PRODUCTNDC                        93238 non-null  object 
 2   PRODUCTTYPENAME                   93238 non-null  object 
 3   PROPRIETARYNAME                   93232 non-null  object 
 4   PROPRIETARYNAMESUFFIX             10163 non-null  object 
 5   NONPROPRIETARYNAME                93234 non-null  object 
 6   DOSAGEFORMNAME                    93238 non-null  object 
 7   ROUTENAME                         91306 non-null  object 
 8   STARTMARKETINGDATE                93238 non-null  int64  
 9   ENDMARKETINGDATE                  4323 non-null   float64
 10  MARKETINGCATEGORYNAME             93238 non-null  object 
 11  APPLICATIONNUMBER                 80141 non-null  object 
 12  LABE

In [7]:
product_df.select_dtypes(include='float64').describe()

,ENDMARKETINGDATE,LISTING_RECORD_CERTIFIED_THROUGH
count,4.323000e+03,8.891300e+04
mean,2.021040e+07,2.020247e+07
std,1.539891e+05,3.294780e+03
min,2.020022e+07,2.020123e+07
25%,2.020081e+07,2.020123e+07
50%,2.020123e+07,2.020123e+07
75%,2.021111e+07,2.020123e+07
max,3.031021e+07,2.021123e+07


In [8]:
product_df.select_dtypes(include='int64').describe()

,STARTMARKETINGDATE
count,9.323800e+04
mean,2.010696e+07
std,1.131769e+05
min,1.900010e+07
25%,2.009090e+07
50%,2.014122e+07
75%,2.018010e+07
max,2.020021e+07


In [9]:
product_df.select_dtypes(include='object').describe()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG
count,91678,93238,93238,93232,10163,93234,93238,91306,93238,80141,93238,90929,90929,90929,42254,4423,93238
unique,91678,91468,7,32716,4022,16257,134,180,10,10711,6611,8976,8769,2391,1285,4,1
top,52380-4802_84a23d69-ed72-34b2-e053-2a91aa0aa80f,OTC MONOGRAPH FINAL,HUMAN OTC DRUG,Ibuprofen,Maximum Strength,Ibuprofen,TABLET,ORAL,ANDA,part352,REMEDYREPACK INC.,ALCOHOL,10,mg/1,"Corticosteroid [EPC],Corticosteroid Hormone Re...",CII,N
freq,1,111,46172,565,408,947,15442,54704,37490,6563,2201,1720,4617,34831,1420,1802,93238


#### Nombre, taux et types des valeurs manquantes par attribut (product_df)

In [10]:
nb_m = product_df.isnull().sum().sort_values()
ratio_m = (product_df.isnull().sum()/product_df.shape[0]).sort_values()
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)
pd.DataFrame({'Types': product_df[list(manquant.index.values)].dtypes,
              'Nb Unique': [len(pd.Categorical(product_df[_]).categories) for _ in product_df[list(manquant.index.values)].columns],
              'Nb manquants': nb_m,
              'Ratio manquants%': ratio_m,})

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTNDC,object,91468,0,0.000000
PRODUCTTYPENAME,object,7,0,0.000000
DOSAGEFORMNAME,object,134,0,0.000000
STARTMARKETINGDATE,int64,7262,0,0.000000
NDC_EXCLUDE_FLAG,object,1,0,0.000000
MARKETINGCATEGORYNAME,object,10,0,0.000000
LABELERNAME,object,6611,0,0.000000
NONPROPRIETARYNAME,object,16257,4,0.000043
PROPRIETARYNAME,object,32716,6,0.000064
PRODUCTID,object,91678,1560,0.016731


___________________________

## package2.csv --> package_df

* __ProductID__  Text/string - concatenation of the NDCproduct code and SPL documentID
* __ProductNDC__  Text/string. - The labeler code and product code segments of the National Drug Code number, separated by a hyphen 
* __NDCPackageCode__ Text/string - The labeler code, product code, and package code segments of the National Drug Code number, separated by hyphens.
* __PackageDescription__   Text/string - A description of the size and type of packaging in sentence form.
* __NDC_Exclude_Flag__ Text/String. (1, object) - (_Y_, _N_, _E_, _I_) This indicates whether the PACKAGE has been removed/excluded from the NDC Directory for failure to respond to FDA’s requests
    * for correction to deficient or non-compliant submissions (_Y_)
    * the listing certification is expired (_E_)
    * the listing data was inactivated by FDA (_I_)
    * The PACKAGE.XLS and PACKAGE.TXT files only contain listing records where NDC_EXCLUDE_FLAG={ _N_ } **
    * PACKAGES_EXCLUDED.XLS and PACKAGES_EXCLUDED.TXT file contains all listing records with an NDC_EXCLUDE_FLAG={ _Y_, _E_, _I_ }.
* __SAMPLE_PACKAGE__ Text/String. (2, object) - This indicates if the package is to be distributed as a sample package. Values={ _Y_, _N_ }

### Remarques

**ProductNDC** est un sous-ensemble de **NDCPackageCode**

**NDC_Exclude_Flag** semble non essentiel puisque nous traitons que de ceux qui sont approuvés.

________________________

In [11]:
print("package_df shape: ", package_df.shape)
print("# of rows: ", package_df.shape[0])
print("# of attributes: ", package_df.shape[1])

package_df shape:  (173887, 8)
# of rows:  173887
# of attributes:  8


In [12]:
package_df.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


In [13]:
package_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173887 entries, 0 to 173886
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   PRODUCTID           173887 non-null  object 
 1   PRODUCTNDC          172387 non-null  object 
 2   NDCPACKAGECODE      171541 non-null  object 
 3   PACKAGEDESCRIPTION  173887 non-null  object 
 4   STARTMARKETINGDATE  173887 non-null  int64  
 5   ENDMARKETINGDATE    6456 non-null    float64
 6   NDC_EXCLUDE_FLAG    173887 non-null  object 
 7   SAMPLE_PACKAGE      173887 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.6+ MB


In [14]:
package_df.select_dtypes(include=['float64']).describe()

,ENDMARKETINGDATE
count,6.456000e+03
mean,2.020787e+07
std,1.055415e+04
min,2.020022e+07
25%,2.020080e+07
50%,2.020123e+07
75%,2.021111e+07
max,2.039083e+07


In [15]:
package_df.select_dtypes(include=['int64']).describe()

,STARTMARKETINGDATE
count,1.738870e+05
mean,2.012930e+07
std,3.844789e+05
min,1.900010e+07
25%,2.010060e+07
50%,2.015110e+07
75%,2.018050e+07
max,2.997122e+07


In [16]:
package_df.select_dtypes(include=['object']).describe()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
count,173887,172387,171541,173887,173887,173887
unique,93084,91080,171447,173885,1,2
top,73069-270_d59de151-be8e-4a5b-82b9-2f9af03d3785,73069-100,20211201,4 CARTON in 1 KIT (45802-929-49) > 1 KIT in 1...,N,N
freq,72,72,43,2,173887,173223


#### Nombre, taux et types des valeurs manquantes par attribut (package_df)

In [17]:
nb_m = package_df.isnull().sum().sort_values()
ratio_m = (package_df.isnull().sum()/package_df.shape[0]).sort_values()
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)
pd.DataFrame({'Types': package_df[list(manquant.index.values)].dtypes,
              'Nb Unique': [len(pd.Categorical(package_df[_]).categories) for _ in package_df[list(manquant.index.values)].columns],
              'Nb manquants': nb_m,
              'Ratio manquants%': ratio_m,})

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTID,object,93084,0,0.000000
PACKAGEDESCRIPTION,object,173885,0,0.000000
STARTMARKETINGDATE,int64,7401,0,0.000000
NDC_EXCLUDE_FLAG,object,1,0,0.000000
SAMPLE_PACKAGE,object,2,0,0.000000
PRODUCTNDC,object,91080,1500,0.008626
NDCPACKAGECODE,object,171447,2346,0.013492
ENDMARKETINGDATE,float64,767,167431,0.962872


# 2.

Listez toutes les relations/règles observées entre les attributs (informations communes, corrélations, chaînes de caractéres communes, attribut inclus dans un autre, ordre des valeurs) ;


## *** Envisageable de faire une analyse de corrélation de valeurs nominales ?

## Informations Communes

* **ProductID**, **ProductNDC**, **StartMarketingDate**, **EndMarketingDate**, **NDC_Exclude_Flag** (_product_ & _package_) - Ces variables sont partagées par les deux tables. Selon la documentation, il est préférable de joindre les tables par l'entremise de la variable __ProductID__ considérant que celle-ci a été ajustée pour empêcher la duplication de ligne. 


## Corrélations

* **Pharm_Classes** - Cette variable devrait démontrer une corrélation avec la variable __SubstanceName__ puisque la documentation les décris comme étant reliées;

* **DosageFormName** & **RouteName** - Ceux-ci devraient démontrer une corrélation puisque l'un (__DosageFormName__) représente les modalités d'administration des médicaments et l'autre (__RouteName__) représente textuellement la manière dont le médicament sera transmis;

* Considérant les informations spécifiques au contenant inclues dans __PackageDescription__ (_package_), il est envisageable que ceux-ci soient corrélés aux __DosageFormName__ et __RouteName__ qui ont trait à la manière d'administrer le médicament.


## Chaines de caractères communes

* __PackageDescription__ (_package_) - Cette variable implique plusieurs informations sous la forme d'une phrase. On retrouve notamment l'information de __NDCPackageCode__ ainsi que des informations d'unités de mesure et des types de contenant.


## Attribut inclus dans un autre

* **ProductID** (_product_ & _package_) - Implique la concaténation de __ProductNDC__ et un identifiant _SPL_. Cette combinaison permet de prévenir la duplication de lignes lorsque les deux tables seront jointes. 
 
* **ApplicationNumber** (_product_) - Représente majoritairement la concaténation entre __MarketingCategoryName__ et ce qui semble être le numéro de série associé au produit;

* __NDCPackageCode__ (_package_) - Contient les informations de la variable __ProductNDC__ ainsi qu'un code représentant le type de paquet utilisé; cette dernière information pourrait être pertinente;


## Ordre des valeurs

* **StrengthNumber** & **StrengthUnit** (_product_) - Il est à considérer que l'utilisation des valeurs de la variable __StrenghtNumber__ doivent être joint de __StrenghtUnit__. N'ayant pas tous la même unité de mesures, ceux-ci devront être convertis sous une seule et même unité de mesure pour fin de bonne analyse.
 
 



# 3.

Détectez et corrigez les incohérences entre des valeurs d’attributs dans les deux tables ; pour
chaque règle identifiée à la question précédente, détectez et corrigez les cas où la règle n’est
pas respectée ;

### 3.1
* Les valeurs des attributs __StartMarketingDate__, __EndMarketingDate__ et __Listing_Record_Certified_Through__ peuvent être transformées en format _date_.


In [18]:
product_df['STARTMARKETINGDATE'] = pd.to_datetime(product_df['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
product_df['ENDMARKETINGDATE'] = pd.to_datetime(product_df['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
product_df['LISTING_RECORD_CERTIFIED_THROUGH'] = pd.to_datetime(product_df['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')

In [19]:
# Si le produit est venu à échéance, il ne devrait pas y avoir de valeur pour la variable LISTING_RECORD_CERTIFIED_THROUGH
print("(product) Nombre de produits ayant atteint leur date de fin affichant toujours une date d'expiration: {0}".format(product_df[product_df['ENDMARKETINGDATE'].notna()]['LISTING_RECORD_CERTIFIED_THROUGH'].notna().sum()))

(product) Nombre de produits ayant atteint leur date de fin affichant toujours une date d'expiration: 0


In [20]:
(product_df['STARTMARKETINGDATE'] > product_df['LISTING_RECORD_CERTIFIED_THROUGH']).sum()

0

In [21]:
package_df['STARTMARKETINGDATE'] = pd.to_datetime(package_df['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
package_df['ENDMARKETINGDATE'] = pd.to_datetime(package_df['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')

In [22]:
# La date de début de marketing (STARTMARKETINGDATE) devrait précéder la date de fin de production (ENDMARKETINGDATE) 
# ou celle d'expiration (LISTING_RECORD_CERTIFIED_THROUGH).
print("(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): {0}".format((product_df['STARTMARKETINGDATE'] > product_df['ENDMARKETINGDATE']).sum()))
print("(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date d'expiration (LISTING_RECORD_CERTIFIED_THROUGH): {0}".format((product_df['STARTMARKETINGDATE'] > product_df['LISTING_RECORD_CERTIFIED_THROUGH']).sum()))
print("(package) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): {0}".format((package_df['STARTMARKETINGDATE'] > package_df['ENDMARKETINGDATE']).sum()))


(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): 0
(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date d'expiration (LISTING_RECORD_CERTIFIED_THROUGH): 0
(package) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): 0


### 3.2
* Les variables _ProductId_, _ProductNDC_ et NDCPackageCode suivent un format précis. Ceux-ci doivent être analysés afin de déterminer s'il y a des formats non acceptables.

    - PRODUCTNDC doit répondre à une structure de digits telle que {3-5}, {3-4}, {4-4}, {4-5}.
    - PRODUCTID concatène la valeur du PRODUCTNDC et un identifiant SPL séparé par un '_'.
    - NDCPACKAGECODE concatène la valeur du PRODUCTNDC et un code segment de 2 digits séparé par '-'.


In [23]:
ndc_pkg_code_wrong = package_df['NDCPACKAGECODE'].str.split('-').apply(lambda x: len(x) if isinstance(x, list) else 0).copy()
ndc_pkg_code_wrong_index = ndc_pkg_code_wrong[(ndc_pkg_code_wrong < 3) & (ndc_pkg_code_wrong > 0)].index

In [24]:
print("Il y a un total de {0} valeurs incohérentes pour la variable NDCPACKAGECODE de la table package".format(package_df.loc[ndc_pkg_code_wrong_index, 'NDCPACKAGECODE'].shape[0]))

Il y a un total de 154 valeurs incohérentes pour la variable NDCPACKAGECODE de la table package


Puisque nous traitons les valeurs manquantes dans la prochaine section et qu'il est possible de déterminer le _NDCPackageCode_ à l'aide de la variable _PackageDescription_, la valeur NaN sera attribué à ces valeurs incohérentes.

In [25]:
package_df.loc[ndc_pkg_code_wrong_index, 'NDCPACKAGECODE'] = np.nan

### 3.3

[...]

In [26]:
# Il semble y avoir mauvaise attribution de la nomenclature
set(product_df[product_df['MARKETINGCATEGORYNAME'].isin(['OTC MONOGRAPH FINAL', 'OTC MONOGRAPH NOT FINAL'])]['APPLICATIONNUMBER'])

{'333D',
 'part',
 'part331',
 'part332',
 'part333',
 'part333A',
 'part333B',
 'part333C',
 'part333D',
 'part333E',
 'part334',
 'part335',
 'part336',
 'part338',
 'part340',
 'part341',
 'part341,part348',
 'part343',
 'part344',
 'part346',
 'part347',
 'part348',
 'part349',
 'part349B',
 'part350',
 'part352',
 'part355',
 'part355B',
 'part356',
 'part356,part355',
 'part357',
 'part357B',
 'part357I',
 'part358',
 'part358A',
 'part358B',
 'part358D',
 'part358F',
 'part358G',
 'part358H'}

In [27]:
# Confirmation que les produits non approuvés n'ont pas de numéro d'application associé.
product_df[product_df['MARKETINGCATEGORYNAME'].isin(['UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE', 'UNAPPROVED DRUG OTHER', 'UNAPPROVED HOMEOPATHIC', 'UNAPPROVED MEDICAL GAS'])]['APPLICATIONNUMBER'].notna().sum()

0

In [28]:
# SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH, ACTIVE_INGRED_UNIT sont des variables à valeurs multiples. Celles-ci, lorsque multiple, sont listées dans le même ordre.
# Confirmation du même nombre d'éléments dans ces trois variables.

mv_errors = []
for index, row in product_df.iterrows():
    #row['SUBSTANCENAME'].str.split(';'). == row['ACTIVE_NUMERATOR_STRENGTH'] == row['ACTIVE_INGRED_UNIT']
    _name_count = len(row['SUBSTANCENAME'].split(';')) if isinstance(row['SUBSTANCENAME'], str) else 0
    _strength_count = len(row['ACTIVE_NUMERATOR_STRENGTH'].split(';')) if isinstance(row['ACTIVE_NUMERATOR_STRENGTH'], str) else 0
    _unit_count = len(row['ACTIVE_INGRED_UNIT'].split(';')) if isinstance(row['ACTIVE_INGRED_UNIT'], str) else 0
    
    if _name_count == _strength_count == _unit_count:
        continue
    else:
        mv_errors.append(index)
    

In [29]:
mv_errors

[90536]

In [30]:
product_df.loc[90536]

PRODUCTID                              72582-001_8123ebc8-3a6e-0c68-e053-2a91aa0a4a6d
PRODUCTNDC                                                                  72582-001
PRODUCTTYPENAME                                                        HUMAN OTC DRUG
PROPRIETARYNAME                                                                 P3 AI
PROPRIETARYNAMESUFFIX                                                             NaN
NONPROPRIETARYNAME                                                              P3 AI
DOSAGEFORMNAME                                                                    GEL
ROUTENAME                                                                     TOPICAL
STARTMARKETINGDATE                                                2019-03-31 00:00:00
ENDMARKETINGDATE                                                                  NaT
MARKETINGCATEGORYNAME                                         OTC MONOGRAPH NOT FINAL
APPLICATIONNUMBER                                     

## 4.

Complétez au maximum les données manquantes dans les deux tables ;

### Product

In [31]:
# Complète la variable PRODUCTID à l'aide d'un mapping provenant de la table package

prodIdMissing_mask = product_df['PRODUCTID'].isna()
packageID_map = package_df[['PRODUCTID', 'PRODUCTNDC']].set_index('PRODUCTNDC').drop_duplicates()

# Apply mapping from package_df
product_df.loc[prodIdMissing_mask, 'PRODUCTID'] = product_df.loc[prodIdMissing_mask, 'PRODUCTNDC'].map(
    packageID_map['PRODUCTID'].to_dict()).values

### Package

In [32]:
pkg_mask = package_df['NDCPACKAGECODE'].isna()
package_df.loc[pkg_mask, 'NDCPACKAGECODE'] = package_df.loc[pkg_mask, 'PACKAGEDESCRIPTION'].str.extract(r'\((.*?)\)').values

In [33]:
prod_mask = package_df['PRODUCTNDC'].isna()
package_df.loc[prod_mask, 'PRODUCTNDC'] = package_df['NDCPACKAGECODE'].str.split('-').apply(lambda x: '-'.join(x[:2]))

## 5.

Détectez et retirez les objets dupliqués dans les deux tables ;

### Possibilité de données dupliquées

#### Product

* PRODUCTID - Doit être unique puisqu'il représente le produit dans la table _product_

#### Package

* NDCPACKAGECODE - Doit être unique puisqu'il représente le paquet dans la table _package_

In [34]:
print("Nombre de code produit en double: ", product_df['PRODUCTID'].duplicated().sum())
print("Nombre de code paquet en double: ", package_df['NDCPACKAGECODE'].duplicated().sum())


Nombre de code produit en double:  0
Nombre de code paquet en double:  2


In [35]:
duplicated_pkg_code = package_df[package_df['NDCPACKAGECODE'].duplicated()]['NDCPACKAGECODE'].values
first_duplicated = package_df[package_df['NDCPACKAGECODE'] == duplicated_pkg_code[0]].copy()
second_duplicated = package_df[package_df['NDCPACKAGECODE'] == duplicated_pkg_code[1]].copy()

#### Premier doublons

In [36]:
first_duplicated

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
53622,45802-929_4290e001-c03c-4bde-a132-5203cc57afb4,45802-929,45802-929-49,4 CARTON in 1 KIT (45802-929-49) > 1 KIT in 1...,2018-08-02,NaT,N,N
53623,45802-929_b2a5b110-7537-4ffc-bf0a-f0fe13379c9d,45802-929,45802-929-49,4 CARTON in 1 KIT (45802-929-49) > 1 KIT in 1...,2018-08-02,NaT,N,N


On remarque qu'il y a différence au niveau du _PRODUCTID_. Ceux-ci semblent faire référence à deux produits différents. <br>


In [37]:
product_df[product_df['PRODUCTID'].isin(first_duplicated['PRODUCTID'])]

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
31495,45802-929_4290e001-c03c-4bde-a132-5203cc57afb4,45802-929,HUMAN PRESCRIPTION DRUG,Mesalamine,NaN,Mesalamine,KIT,NaN,2018-08-02,NaT,ANDA,ANDA076751,Perrigo New York Inc,NaN,NaN,NaN,NaN,NaN,N,2020-12-31
31496,45802-929_b2a5b110-7537-4ffc-bf0a-f0fe13379c9d,45802-929,HUMAN PRESCRIPTION DRUG,Mesalamine,NaN,Mesalamine,KIT,NaN,2018-08-02,NaT,ANDA,ANDA076751,Perrigo New York Inc,NaN,NaN,NaN,NaN,NaN,N,2020-12-31


On remarque qu'à l'exception du _PRODUCTID_, cela semble être le même produit. De ce fait, nous allons en retirer un dans la table _product_ et son équivalent dans la table _package_.

In [38]:
first_removed_duplicated_id = first_duplicated['PRODUCTID'].values[1]

In [39]:
product_df = product_df[product_df['PRODUCTID'] != first_removed_duplicated_id]
package_df = package_df[package_df['PRODUCTID'] != first_removed_duplicated_id]

#### Deuxième doublons

In [40]:
second_duplicated

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
56138,47593-359_a406ac88-de15-4494-88e6-bd0f3dc77793,47593-359,47593-359-41,"750 mL in 1 BOTTLE, PLASTIC (47593-359-41)",2002-07-26,NaT,N,N
56139,47593-359_a406ac88-de15-4494-88e6-bd0f3dc77793,47593-359,47593-359-41,"750 mL in 1 BOTTLE, PLASTIC (47593-359-41)",2013-09-05,NaT,N,N


On remarque qu'il y a différence au niveau du _STARTMARKETINGDATE_. Ceux-ci font référence à deux dates différentes. Afin de confirmer laquelle des lignes est la bonne, nous allons voir dans la table _product_ pour confirmer quelle date est la bonne.

In [41]:
print("La date de référence est le {0}".format(str(product_df[product_df['PRODUCTID'].isin(second_duplicated['PRODUCTID'])]['STARTMARKETINGDATE'].values[0])))

La date de référence est le 2002-07-26T00:00:00.000000000


De ce fait, nous allons retirer la ligne faisant référence à l'année de départ 2013-09-05

In [42]:
package_df = package_df[package_df['STARTMARKETINGDATE'] != second_duplicated['STARTMARKETINGDATE'].values[1]]

## 6.

Intégrez les deux tables et nettoyez le résultat (données dupliquées, incomplètes, incohérentes, erronées) ;


In [43]:
merged_table = pd.merge(product_df, package_df, on='PRODUCTID', suffixes=('_product', '_package'))

In [44]:
nb_m = merged_table.isnull().sum().sort_values()
ratio_m = (merged_table.isnull().sum()/merged_table.shape[0]).sort_values()
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)
pd.DataFrame({'Types': merged_table[list(manquant.index.values)].dtypes,
              'Nb Unique': [len(pd.Categorical(merged_table[_]).categories) for _ in merged_table[list(manquant.index.values)].columns],
              'Nb manquants': nb_m,
              'Ratio manquants%': ratio_m,})

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTID,object,93079,0,0.000000
PACKAGEDESCRIPTION,object,173876,0,0.000000
NDCPACKAGECODE,object,173876,0,0.000000
PRODUCTNDC_package,object,91693,0,0.000000
NDC_EXCLUDE_FLAG_product,object,1,0,0.000000
NDC_EXCLUDE_FLAG_package,object,1,0,0.000000
LABELERNAME,object,6603,0,0.000000
MARKETINGCATEGORYNAME,object,10,0,0.000000
SAMPLE_PACKAGE,object,2,0,0.000000
DOSAGEFORMNAME,object,134,0,0.000000


On remarque qu'il y a 4 variables traitant des mêmes sujets dans les deux tables:

Nous déterminerons les variables en fonction de leur complétude vis-à-vis la table intégrée.

* NDC_EXCLUDE_FLAG - Aucune valeur manquante. Nous en garderons simplement une;
* PRODUCTNDC - Aucune valeur manquante. Nous en garderons simplement une;
* STARTMARKETINGDATE - On constate que la variable provenant de la table _package_ affiche des valeurs manquantes. Nous retirerons donc *STARTMARKETINGDATE_package*;
* ENDMARKETINGDATE - On constate qu'il y a moins de données manquantes provenant de la table _package_. Nous retirerons donc *ENDMARKETINGDATE_product*;

In [45]:
merged_table.drop(columns=['PRODUCTNDC_package', 'NDC_EXCLUDE_FLAG_package',
                           'STARTMARKETINGDATE_package', 'ENDMARKETINGDATE_product'],
                  inplace=True)

merged_table.rename(columns={'PRODUCTNDC_product': 'PRODUCTNDC',
                             'NDC_EXCLUDE_FLAG_product': 'NDC_EXCLUDE_FLAG',
                             'STARTMARKETINGDATE_product': 'STARTMARKETINGDATE',
                             'ENDMARKETINGDATE_package': 'ENDMARKETINGDATE'}, inplace=True)

## 7.

Proposez un nouvel ensemble d’attributs (représentation) qui élimine la redondance des informations dans les valeurs des attributs, et qui permet de transformer l’attribut PHARM_CLASSES
en un ensemble d’attributs distincts correspondant à ses différents champs EPC, CS, MOA,
PE etc. ;


## 8.

À partir de la nouvelle représentation, proposez un ensemble d’attributs à utiliser pour
prédire le plus précisément possible toutes les classes pharmacologiques établies d’un médicament (champ EPC dans l’attribut PHARM_CLASSES) ;


## 9.

En se basant sur la réduction de dimension obtenue à la question précédente, appliquez un
modèle de classification pour prédire les classes pharmacologiques établies des médicaments
pour lesquels l’information est manquante ;
